In [1]:
import math
import pickle

import tqdm
import mlflow
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

from utils import *

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о
составах команд и повопросных результатах (поле mask в results.pkl). Для унификации
предлагаю:

In [2]:
with open('chgk/players.pkl', 'rb') as f:
    players = pickle.load(f)
    
with open('chgk/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [3]:
train_ids = get_ids_by_date([2019], tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [4]:
print(f"Количество турниров для обучения {len(train_ids)}, количество турниров для теста {len(test_ids)}")

Количество турниров для обучения 659, количество турниров для теста 165


In [5]:
# Функции для рассчета коэффициента корреляции Спирмена и Кендалла

def spearman(pred_ratings, actual_ratings):
    corrs = []
    for pr, tr in zip(pred_ratings, actual_ratings):
        corrs.append(stats.spearmanr(pr, tr).correlation)
    index = ~np.isnan(corrs) * 1
    corrs = np.array(corrs)[index]
    corr_coef = np.mean(corrs)
    print('Spearman', corr_coef)
    return corr_coef
    
def kendall(pred_ratings, actual_ratings):
    corrs = []
    for pr, tr in zip(pred_ratings, actual_ratings):
        corrs.append(stats.kendalltau(pr, tr).correlation)
    index = ~np.isnan(corrs) * 1
    corrs = np.array(corrs)[index]
    corr_coef = np.mean(corrs)
    print('Kendall', corr_coef)
    return corr_coef
    
def rating_to_score(ratings):
    positions = []
    sorted_r = sorted(ratings)
    for rating in ratings:
        positions.append(sorted_r.index(rating) + 1)
    return positions

def calc_correlations(ids, results, players_mean_mu):
    pred_ratings = []
    actual_ratings = []
    for id_ in ids:
        tournament = results[id_]
        t_pred_ratings = []
        t_actual_ratings = []
        for team in tournament:
            scores = []
            for member in team['teamMembers']:
                member_id = member['player']['id']
                if member_id in players_mean_mu:
                    scores.append(players_mean_mu[member_id])
            t_actual_ratings.append(team['position'])
            if len(scores):
                index = ~np.isnan(scores)
                scores = list(np.array(scores)[index])
                if len(scores) == 0:
                    t_pred_ratings.append(0)
                else:
                    prob = -np.mean(scores)
                    t_pred_ratings.append(prob)
            else:
                t_pred_ratings.append(0)
        pred_ratings.append(rating_to_score(t_pred_ratings))
        actual_ratings.append(t_actual_ratings)
    
    s_corr = spearman(pred_ratings, actual_ratings)
    k_corr = kendall(pred_ratings, actual_ratings)
    return s_corr, k_corr

2. Постройте baseline-модель на основе линейной или логистической регрессии, которая
будет обучать рейтинг-лист игроков. 

Проанализировав Ваши замечания и подсказки, я решил (на свой риск и страх) сделать бэйзлайн модель не на основе логистической, либо линейной регрессии, а сделать итерационный алгоритм, правда ничем неподкрепленный, но мне он показался "напрашивающимся".

Так как ответ игрока на какой-либо вопрос $x$ это "верно" либо "неверно", я сделал предположение, что сила какого-либо игрока это параметр $mu_k$ распределения Бернулли, и он зависит от сложности турнира $w_t$:


Вероятность игрока k правильно ответит на вопрос турнира t:

$p(x=1|k, t) = mu_{kt}$

По методу максимального правдоподобия, параметр распределения Бернулли равен отношению количества правильных ответов к количеству всех вопросов: 
        
$mu_{k}$ = $\frac{1}{N}$$\sum_{i=1}^{N} I(x_i)$

где $I(x_i)$ индикатор, равен 1 если игрок ответил правильно но вопрос $x_i$, и 0 в обратном случае

Но проблема в том, что каждый турнир имеет разную сложность и вот так просто найти силу игрока не получится.
Поэтому я ввел параметр $w_t$ - сложность турнира:

$\sum_{t=1}^{T} w_t = 1$


$mu_{mle}$ для игрока k на турнира t равно ($N_t$ - количество вопросов на турнире t):
        
1. $mu_{kt}$ = $\frac{1}{N_t}$$\sum_{i=1}^{N_t} I(x_i)$

А полная вероятность равна (T количество турниров игрока k):
        
2. $mu_{k}$ = $\sum_{t=1}^{T} mu_{kt} * w_{t}$

Возникает вопрос чему равен $w_t$. Например в футболе, для меня лично, сложность турнира определяется количеством сильных команд. Например, лига чемпионов выше уровнем чем лига европы, потому что в 1-турнире участвуют больше грандов.

Поэтому $w_t$ определим как среднюю силу всех игроков, участвовавших на турнире t ($N_p$ - количество участников на турнире t):

3. $w_t$ = $\frac{1}{N_p}$$\sum_{k=1}^{N_p} mu_{k}$

И так получилось следующее:
1. Инициализируем $w_t$ случайными значениями от 0 до 1
2. Находим $mu_{k}$ по формуле 2
3. Рассчитываем $w_t$ по формуле 3
4. Повторяем 2-3 шаги до схождения

In [6]:
class BaselineModel:
    def __init__(self, train_ids, results):
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        self.mu_vals = create_nested_dict(train_ids, results)
        
        """Инициализируем сложность турниров случайными значениями"""
        comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        self.consider_questions_num = False
        
        self.train_ids = train_ids
        self.results = results
        
    def calc_players_strength_for_competition(self, comp_id, weight, eps=1e-5):
        """Расчитываем формулу 2 для каждого игрока на турнире comp_id"""
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 0 and len(team['teamMembers']) != 0:
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    new_mu = np.mean(mask)
                    if self.consider_questions_num:
                        penalty = np.sqrt(np.log10(self.player_appereances[player_id] + 1))
                        self.players_mu[player_id].append(new_mu * weight * penalty)                
                    else:
                        self.players_mu[player_id].append(new_mu * weight)
                        
    def get_comp_prob(self, comp_id):
        """Рассчитываем формулу 3 для турнира comp_id"""
        comp = self.mu_vals[comp_id]
        scores = []
        max_mu = 1e-5
        for team in comp:
            for player in comp[team]:
                comp[team][player] = np.mean(self.players_mu[player])
                scores.append(comp[team][player])
                if comp[team][player] > max_mu:
                    max_mu = comp[team][player]
        if len(scores):
            return np.mean(scores), max_mu
        return 1e-5, max_mu
    
    def get_comp_probs(self):
        """Рассчитываем формулу 3 для всех турниров и нормализуем сложность турнира и силу игроков"""
        gl_max_mu = 1e-5
        for comp_id in tqdm.tqdm(self.mu_vals):
            prob, max_mu = self.get_comp_prob(comp_id)
            self.comp_probs[comp_id] = prob
            if max_mu > gl_max_mu:
                gl_max_mu = max_mu
        norm = np.array(list(self.comp_probs.values())).sum()
        for comp_id in self.mu_vals:
            self.comp_probs[comp_id] = self.comp_probs[comp_id] / (norm + 1e-4)
        for player_id in self.players_mu:
            self.players_mu[player_id] = list(np.array(self.players_mu[player_id]) / gl_max_mu)
            self.players_mean_mu[player_id] = np.mean(self.players_mu[player_id])
    
    def iteration(self):
        """Запускаем итерацию 2-3 пунктов"""
        self.players_mu = create_players_dict(self.train_ids, self.results)
        for comp_id in self.comp_probs:
            self.calc_players_strength_for_competition(comp_id, self.comp_probs[comp_id])
        self.get_comp_probs()

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров.
Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь
неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны
эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными
составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые
корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между
реальным ранжированием в результатах турнира и предсказанным моделью,
усреднённые по тестовому множеству турниров 1 .

Обучим бэйзлайн модель

In [7]:
model = BaselineModel(train_ids, results)
experiment_id = mlflow.set_experiment("baseline-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(30):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:03<00:00, 167.53it/s]


Iteration 1
Spearman 0.7220747895307745
Kendall 0.5446669588972131




100%|██████████| 659/659 [00:03<00:00, 164.81it/s]


Iteration 2
Spearman 0.762340492270467
Kendall 0.5874501656938121




100%|██████████| 659/659 [00:04<00:00, 164.25it/s]


...

Iteration 28
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:04<00:00, 163.18it/s]


Iteration 29
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:04<00:00, 163.27it/s]


Iteration 30
Spearman 0.7957590406894631
Kendall 0.6188709517282583




Видим, что после 22-итерации метрики сошлись и не меняются в последующих итерациях. Взглянем на топ-100 игроков (имя знатока, его сила mu и количество сыгранных вопросов):

In [8]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Роман Петров 1.0 30
2 Игорь Петров 1.0 30
3 Андриан Иоаннидис 1.0 30
4 Дмитрий Луконин 1.0 30
5 Анатолий Королихин 1.0 30
6 Евгений Чернецкий 0.888888888888889 30
7 Александр Усов 0.888888888888889 30
8 Дамир Фёдоров 0.888888888888889 30
9 Василий Марков 0.888888888888889 30
10 Константин Карпов 0.611111111111111 30
11 Иван Карпов 0.611111111111111 30
12 Виктория Кулёва 0.611111111111111 30
13 Яна Кулёва 0.611111111111111 30
14 Юлия Крюкова 0.04872634640175428 36
15 Наталья Артемьева 0.04872634640175428 36
16 Екатерина Горелова 0.04872634640175428 36
17 Антон Калинин 0.04720364807669947 36
18 Ольга Остросаблина 0.04499745956054219 36
19 Валентина Подюкова 0.044305169286235076 36
20 Махбуба Мамаджанова 0.04415825142658982 36
21 Дарина Калнина 0.042635553101535 36
22 Артём Улюкин 0.042635553101535 36
23 Мария Каменских 0.04194766286883021 72
24 Сергей Пашкевич 0.04118546334279094 36
25 Анна Щеголькова 0.03959015645142535 36
26 Матвей Конюхов 0.03959015645142535 36
27 Иван Конюхов 0.039

В топе рейтинга игроков действительно ноунеймы, сыгравшие мало вопросов. Также в рейтинге есть игроки и с большим количеством сыгранных вопросов, и вроде бы все они действительно топы. А теперь взглянем на топ-100 турниров по сложности

In [9]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Чемпионат Кипра среди школьников 0.06969656127281265
2 Чемпионат Мира. Финал. Группа А 0.0025482793296085077
3 Чемпионат Мира. Этап 2. Группа А 0.0025253178719369406
4 Чемпионат Мира. Этап 3. Группа А 0.002521323970887555
5 Чемпионат Мира. Этап 1. Группа А 0.002335401018211983
6 Чемпионат Санкт-Петербурга. Высшая лига 0.0021585229019216
7 Чемпионат России 0.002130261684370819
8 Чемпионат Мира. Финал. Группа В 0.0021014379371368072
9 Чемпионат Мира. Этап 3. Группа В 0.0020964432094983812
10 Славянка 0.002089387674089496
11 Чемпионат Мира. Этап 2. Группа В 0.0020804532820144516
12 Весна в ЛЭТИ 0.002014719708671475
13 Мемориал Дмитрия Коноваленко 0.002007096183240122
14 Второй тематический турнир имени Джоуи Триббиани 0.002004998508675578
15 Мемориал Дмитрия Мотрича 0.0019985787067136984
16 Год театра 0.001993544602803904
17 Чемпионат Мира. Этап 1. Группа В 0.0019777875019953255
18 Регулярный чемпионат МГУ. Высшая лига. Третий игровой день 0.0019749948833337957
19 Знатокиада. Всеобщий О

На первом месте чемпионат среди школьников, видимо те ноунеймы и подняли рейтинг этого турнира, далее идут разные этапы чемпионата мира и чемпионат России, вроде более-менее правильно

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько
игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать
упрощающие предположения, но теперь переменные “игрок X ответил на вопрос
Y” при условии данных должны стать зависимыми для игроков одной и той же
команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем
растут (скорее всего, ненамного, но расти должны), выберите лучшую модель,
используя целевые метрики.

Рассмотрим правдоподобие результата $X$ какой-либо команды $m$ на каком-либо турнире $t$ если в команде только один игрок:

$p(X_{tm}|mu_m, t) = \prod_{i=1}^{N} mu_m^{x_i} * (1 - mu_m) ^ {1 - x_i}$

Но в командах обычно больше чем один игрок. Нам неизвестно кто из игроков ответил на определенный вопрос. Вообще говоря бывают разные случаи: бывает все игроки знают ответ, бывает несколько игроков знают и отвечают правильно, бывает несколько знатоков имеют правильную версию, но отвечает игрок с неправильной версией. Смоделировать эти ситуации сложно. Я решил упростить задачу и сделать предположение, что если команда ответила неправильно, то ни у кого не было правильной версии. Дальше я сделал предположение, что на отвеченный вопрос знал ответ только один знаток. Поэтому у меня получилась некая класстеризация вопросов по игрокам. Давайте рассмотрим правдоподобие результата (K количество игроков в команде m, $pi_k$ - равно 1 если k-ый игрок ответил на вопрос, и 0 если нет):

$p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}) = \prod_{i=1}^{N} (\sum_{k=1}^{K} pi_k * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})$

$\sum_{k=1}^{K} pi_k = 1$

Рассмотрим логарифм правдоподобия:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, pi)) = \sum_{i=1}^{N} (log(\sum_{k=1}^{K} pi_{mk} * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i}))$

И как мы видим максимизировать это правдоподобие сложно, поэтому внесем скрытые переменные $z$:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, z)) = \sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

$\sum_{k=1}^{K} z_k = 1$

Теперь распишем правдободобие для всех данных (все турниры T, все команды M, все вопросы N):

$log(X|everything) = \sum_{t=1}^{T}\sum_{m=1}^{M}\sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

Найти ожидание $z_{mkt}$ можно по формуле:

1. $E[z_{mkt}] = \frac{pi_{mkt} * p(X_{tm}|mu_{mk}, t)}{\sum_{j=1}^{K}pi_{mjt} * p(X_{tm}|mu_{mj}, t)} $

Опять же тут у всех турниров разные уровни, поэтому я ввел следующее равенство, вероятность игрока k ответит на вопрос турнира t равно (p_k - сила игрока, w_t - сложность вопроса):

2. $p(x=1|k, t) = \frac{p_k}{p_k + w_t} = mu_{kt}$


Теперь приступим к EM-алгоритму:

- на шаге expectation найдем все $E[z_{mkt}]$
- на шаге максимизации нужно найти значения $pi_{mkt}$, $w_t$, $p_k$. $mu_{kt}$ найдем по формуле 2


9. $pi_{kt} = \frac{1}{N}\sum_{n=1}^{N}E[z_{nkt}]$   (формулы 9.60 из Bishop)

А для нахождения $p_k$ и $w_t$ распишем часть правдободобия, где присутсвуют эти параметры:

$l = \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} * log(\frac{p}{p + w_t}^{x_n} * (1 - \frac{p}{p + w_t})^{1 - x_n}) + ...$

$l = \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} * log(\frac{p_k}{p_k + w}^{x_n} * (1 - \frac{p_k}{p_k + w})^{1 - x_n}) + ...$

Найдем производную по $p$ и попытаемся найти экстремум функции:

3. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \sum_{t=1}^{T}\sum_{n=1}^{N_t} \frac{z_{nt}}{p + w_t} = 0$

4. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \sum_{k=1}^{K}\sum_{n=1}^{N}\frac{z_{nk}}{p_k + w} = 0$

Найти значения $p$ и $w$ сложно, поэтому я использовал scipy.optimize.fsolve

###### Отступление:
Изначально я посчитал формулы 3 и 4 неправильно, как:

5. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \frac{1}{p + w_t}\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt}  = 0$

6. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \frac{1}{p_k + w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} = 0$

И естественно легко максимизировал их, и получил:

7. $p = \frac{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n w_t}{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} - \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n}$


8. $w = \frac{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k - \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k x_n}{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} x_n}$

По формуле 7 было видно, что чем больше отвеченных вопросов у знатока и чем больше сложность турнира, тем больше его сила, и наоборот. А по формуле 8 было заметно, что чем больше неправильных ответов у сильного знатока, тем больше сложность вопроса, все вроде бы логично. НО при расчете этих формул была допущена грубая ошибка, которую я заметил только в последний день дедлайна, когда начал писать чистовую версию. Но эти значения все же использую как estimated value для scipy.optimize.fsolve

In [10]:
def dl_dmu(mu, z_x, znts, weights):
    """правдоподобие по формуле 5"""
    temp = z_x / mu
    for i, znt in enumerate(znts):
        temp -= znt / (weights[i] + mu)
    return temp

In [11]:
def dl_dw(weight, znk, z_x, mus):
    """правдоподобие по формуле 6"""
    result = (np.sum(znk) - np.sum(z_x)) / weight
    for mu, z in zip(mus, znk):
        result -= z / (weight + mu)
    return result

In [12]:
class EMModel:
    def __init__(self, train_ids, results, q=1.5, force=False, cut_num=None):
        self.z_vals = create_nested_dict(train_ids, results)
        self.pi_vals = create_nested_dict(train_ids, results)
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        # comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        comp_probs = np.ones(len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        
        self.train_ids = train_ids
        self.results = results
        self.q = q
        self.force = force
        self.cut_num = cut_num
        
    def calculate(self, func, eps=1e-5):
        for comp_id in tqdm.tqdm(self.comp_probs):
            weight = self.comp_probs[comp_id]
            for team in self.results[comp_id]:
                team_id = team['team']['id']
                mask = team['mask']
                mask = to_int(mask)
                func(team, mask, weight, comp_id, team_id, eps)
        
    def expectation(self, team, mask, weight, comp_id, team_id, eps):
        """Находим ожидание скрытых пременных"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            pis = []
            mus = []
            # Initialize
            for player in team['teamMembers']:
                player_id = player['player']['id']
                pis.append(self.pi_vals[comp_id][team_id][player_id])
                mus.append(self.players_mean_mu[player_id] / (self.q * self.players_mean_mu[player_id] + weight + eps))
                self.z_vals[comp_id][team_id][player_id] = []

            pis = np.array(pis)
            mus = np.array(mus)

            # Expectation
            for answer in mask:
                denominator = pis * (mus ** answer) * (1 - mus) ** (1 - answer)
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    znk = denominator[i] / (np.sum(denominator) + eps)
                    assert znk <= 1, denominator
                    self.z_vals[comp_id][team_id][player_id].append(znk)

    def maxizmize_mus_and_pis_by_team(self, team, mask, weight, comp_id, team_id, eps):
        """Максимизируем pi по формуле 9"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            # Maximization
            for i, player in enumerate(team['teamMembers']):
                player_id = player['player']['id']
                nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                new_mu = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                self.z_x[player_id].append(new_mu)
                self.znt[player_id].append(nk)
                self.w_t[player_id].append(weight)
                self.pi_vals[comp_id][team_id][player_id] = nk / (mask.shape[0] + eps)

    def maximize_player_mus(self):
        """Максимизируем p по формуле 5"""
        for player_id in self.z_x:
            if self.cut_num:
                if len(self.z_x[player_id]) > self.cut_num:
                    self.z_x[player_id] = self.z_x[player_id][-self.cut_num:]
                if len(self.w_t[player_id]) > self.cut_num:
                    self.w_t[player_id] = self.w_t[player_id][-self.cut_num:]
                if len(self.znt[player_id]) > self.cut_num:
                    self.znt[player_id] = self.znt[player_id][-self.cut_num:]
                
            z_x = np.sum(self.z_x[player_id])
            mu = np.sum(np.array(self.z_x[player_id]) * np.array(self.w_t[player_id]))
            estimate_mu = mu / (np.sum(self.znt[player_id]) - np.sum(self.z_x[player_id]))
            new_mu = fsolve(dl_dmu, estimate_mu, args=(z_x, self.znt[player_id], self.w_t[player_id]), xtol=1)[0]
            if self.force:
                k = np.log10(len(self.z_x[player_id]))
                k = pow(k, 0.25)
                new_mu = new_mu * k
            self.players_mean_mu[player_id] = new_mu
            assert new_mu >= 0, (new_mu, mu, z_x, self.znt[player_id], self.w_t[player_id])

    def maxizmize_comp_weights(self, comp_id, eps=1e-5):
        """Максимизируем w по формуле 6"""
        znk_xn = []
        znk = []
        mus = []
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
                # Maximization
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                    summ = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                    znk_xn.append(summ)
                    znk.append(nk)
                    mus.append(self.players_mean_mu[player_id])
        
        znk_uk_sum = np.sum(np.array(znk) * np.array(mus))
        znk_xn_uk_sum = np.sum(np.array(znk_xn) * np.array(mus))
        znk_xn_sum = np.sum(znk_xn)
        estimate_weight = (znk_uk_sum - znk_xn_uk_sum) / (znk_xn_sum + eps)
        self.comp_probs[comp_id] = fsolve(dl_dw, estimate_weight, args=(znk, znk_xn, mus), xtol=1)[0]
        assert self.comp_probs[comp_id] >= 0, self.comp_probs[comp_id]
            
    def maxizmize_comps_weights(self, eps=1e-5):
        for comp_id in self.comp_probs:
            self.maxizmize_comp_weights(comp_id)
            
    def iteration(self):
        self.z_x = create_players_dict(self.train_ids, self.results)
        self.znt = create_players_dict(self.train_ids, self.results)
        self.w_t = create_players_dict(self.train_ids, self.results)
        self.calculate(self.expectation)
        self.calculate(self.maxizmize_mus_and_pis_by_team)
        self.maximize_player_mus()
        self.maxizmize_comps_weights()

Обучим нашу модель:

In [13]:
model = EMModel(train_ids, results, q=1)
experiment_id = mlflow.set_experiment("EM-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(5):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 79.53it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Iteration 1
Spearman 0.7028830613334528
Kendall 0.531021119434825




100%|██████████| 659/659 [00:08<00:00, 76.79it/s] 


Iteration 2
Spearman 0.7618072619818467
Kendall 0.585361334371809




100%|██████████| 659/659 [00:08<00:00, 79.95it/s] 


Iteration 3
Spearman 0.7754113893045769
Kendall 0.6004477256407548




100%|██████████| 659/659 [00:08<00:00, 79.20it/s] 


Iteration 4
Spearman 0.7787656689253655
Kendall 0.6037704853040248




100%|██████████| 659/659 [00:08<00:00, 79.95it/s] 


Iteration 5
Spearman 0.7778726621458432
Kendall 0.6022618096530413




По традиции посмотрим на топов:

In [14]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Елена Бровченко 1760.9610835684846 36
2 Артём Стетой 522.6464941548817 36
3 Максим Пилипенко 327.6640745600486 36
4 Надежда Бирюкова 312.30864277685725 36
5 София Савенко 280.2713018277753 36
6 Павел Линицкий 268.8679441174514 72
7 Валентина Подюкова 256.5815350685322 36
8 Светлана Гусарова 156.00309153406806 111
9 Семён Зайдельман 145.59665639672994 36
10 София Лебедева 126.5927420819299 36
11 Алексей Александров 123.14619286204228 36
12 Елизавета Коваленко 120.30742513188547 36
13 Полина Джегур 117.79332658139283 36
14 Сергей Достовалов 98.21753370623411 36
15 Инга Лоренц 96.94293794865193 36
16 Ольга Остросаблина 83.15890183395261 36
17 Виолетта Глазкова 81.13363427191074 36
18 Сергей Волков 81.1007831186807 77
19 Мария Суханова 79.83425273732931 36
20 Павел Ворончихин 64.45691744774554 72
21 Екатерина Лукьянцева 60.997199855992996 72
22 Сергей Завьялов 58.37803117640056 36
23 Эвелин Бельцер 57.22465452006628 36
24 Андрей Зацаринный 56.23947952514265 36
25 Тимур Мкоян 55.837788792

Видим, что топ полностью забит ноунемами с маленьким количеством сыгранных вопросов

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов.
Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть
сложные вопросы, а на турнирах для школьников — простые)?

In [15]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 6.184188964100192
2 Синхрон высшей лиги Москвы 5.003463197901836
3 Первенство правого полушария 4.496939073144774
4 Воображаемый музей 4.342303154098001
5 Записки охотника 3.957585833909239
6 Знание – Сила VI 3.710961134272077
7 Ускользающая сова 3.66146446545628
8 Зеркало мемориала памяти Михаила Басса 3.468599595483022
9 Чемпионат Мира. Этап 2. Группа В 3.40036367735341
10 Чемпионат России 3.351890716621746
11 Кубок городов 3.341237189888915
12 Чемпионат Минска. Лига А. Тур четвёртый 3.3358342945662836
13 Чемпионат Мира. Этап 2. Группа А 3.2676503622126254
14 Чемпионат Мира. Этап 2 Группа С 3.255407233188397
15 Тихий Донец: омут первый 3.1784601383608675
16 Чемпионат Мира. Этап 3. Группа В 3.173605578399612
17 Львов зимой. Адвокат 3.1419564811388114
18 Чемпионат Мира. Этап 1. Группа А 3.1023583626086575
19 All Cats Are Beautiful 3.0715507221977303
20 Чемпионат Мира. Финал. Группа С 3.060097296947033
21 Серия Premier. Седьмая печать 3.0526088580996937
22 Антибинго 2.9983

###### В топ-20 много этапов чемпионата мира, есть чемпионат России, школьных турниров не вижу, поэтому можно сказать, что рейтинг более-менее соответствует реальности

## 6
Как мы видим в топе игроки, которые сыграли мало игр, попытаемся это исправить.

Для этого придумал 2 подхода:

1. В формулу 2 добавим параметр q > 1 (я назвал его параметром регуляризации) 

$p(x=1|k, t) = \frac{p_k}{q * p_k + w_t} = mu_{kt}$

Как мы видим этот параметр препятствует игроку стать слишком сильным, поэтому это должно сместить игроков ставших сильными засчет малого количества игр

2. Умножить силу игрока на функцию от количества сыгранных вопросов. Но это конечно явное притягивание за уши


- k = np.log10(questions_num)
- k = pow(k, 0.25)
- new_p = p * k

In [16]:
# Попробуем вариант 1
model = EMModel(train_ids, results, q=1.5)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 79.42it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.6948360290027416
Kendall 0.5289979012162374




100%|██████████| 659/659 [00:08<00:00, 77.35it/s] 


Iteration 2
Spearman 0.7834280403646596
Kendall 0.60772195617265




100%|██████████| 659/659 [00:08<00:00, 76.82it/s] 


...

Iteration 8
Spearman 0.8188212471600819
Kendall 0.6421665300094274




100%|██████████| 659/659 [00:09<00:00, 71.75it/s] 


Iteration 9
Spearman 0.8197718816758025
Kendall 0.6434750186954843




100%|██████████| 659/659 [00:08<00:00, 77.45it/s] 


Iteration 10
Spearman 0.82111419654097
Kendall 0.6449986064154261




In [17]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Максим Пилипенко 7.558197050802041 36
2 Максим Руссо 6.991720640664143 2178
3 Дмитрий Кудинов 6.452095622448951 45
4 Михаил Савченков 6.091743033234459 3215
5 Александра Брутер 6.091335603894947 2692
6 Иван Семушин 5.778571478117647 3774
7 Станислав Мереминский 5.534448245528814 1584
8 Артём Сорожкин 5.424092587694764 4849
9 Валентина Подюкова 5.408777900076219 36
10 Евгений Спектор 5.391727254434921 230
11 Михаил Царёв 5.388901844493078 501
12 Сергей Николенко 5.180003008693108 2039
13 Сусанна Бровер 4.817176214761549 600
14 Алексей Гилёв 4.805968402267268 4306
15 Михаил Левандовский 4.805055828788816 1457
16 Анастасия Калинина 4.766714450972062 36
17 Анна Карпелевич 4.719389368203894 60
18 Сергей Волков 4.697685877221429 77
19 Александр Либер 4.68282665317292 3751
20 Игорь Мокин 4.674810205951262 1176
21 Сергей Спешков 4.627448790063038 3737
22 Антон Саксонов 4.592286325404301 1194
23 Сергей Завьялов 4.552054357154051 36
24 Александр Мосягин 4.509480998707694 1112
25 Александр Марк

###### Действительно, топов стало намного больше в рейтинге. Регуляризация помогает. Можно было обучить еще 10 итерации и топы могли бы подняться еще выше, но времени было мало

In [18]:
top_comps = get_top_comps(model.comp_probs, 20, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 6.244245645561327
2 Синхрон высшей лиги Москвы 5.1424666648285395
3 Первенство правого полушария 4.550860074945999
4 Воображаемый музей 4.349318698445566
5 Записки охотника 4.027134927532485
6 Ускользающая сова 3.9856736162127193
7 Знание – Сила VI 3.8025276414735916
8 Зеркало мемориала памяти Михаила Басса 3.748558469304951
9 Чемпионат Минска. Лига А. Тур четвёртый 3.5649243206265164
10 Чемпионат Мира. Этап 2 Группа С 3.5459750742181484
11 Кубок городов 3.4956777934829577
12 Чемпионат Мира. Этап 2. Группа В 3.4212909249859766
13 Чемпионат России 3.4018533703333134
14 Чемпионат Мира. Финал. Группа С 3.3007444769838483
15 Чемпионат Мира. Этап 3. Группа В 3.2811427149877677
16 Чемпионат Мира. Этап 3. Группа С 3.275461770261215
17 Тихий Донец: омут первый 3.2269071120145836
18 Львов зимой. Адвокат 3.2198373002398126
19 All Cats Are Beautiful 3.181608931261602
20 Серия Premier. Седьмая печать 3.170240717525191
21 Чемпионат Мира. Этап 2. Группа А 3.1485419991601105
22 VERSUS: 

###### Рейтинг турниров вроде бы особо не изменился

In [19]:
# Попробуем вариант 2
model = EMModel(train_ids, results, q=1.5, force=True)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 77.31it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7346822273859194
Kendall 0.5623209350902669




100%|██████████| 659/659 [00:08<00:00, 75.05it/s] 


Iteration 2
Spearman 0.806982989702325
Kendall 0.6303589991105477




100%|██████████| 659/659 [00:08<00:00, 76.47it/s] 


...

Iteration 8
Spearman 0.8299351223631827
Kendall 0.6552424696286221




100%|██████████| 659/659 [00:08<00:00, 76.78it/s] 


Iteration 9
Spearman 0.8303311395826822
Kendall 0.6554605510762985




100%|██████████| 659/659 [00:08<00:00, 74.80it/s] 


Iteration 10
Spearman 0.8304130863743456
Kendall 0.6555061241968426




In [20]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Максим Руссо 115.63836332047187 2178
2 Александра Брутер 93.96909560789297 2692
3 Михаил Савченков 89.57574509185143 3215
4 Артём Сорожкин 87.52794090679502 4849
5 Иван Семушин 86.03829265731585 3774
6 Максим Пилипенко 85.61925459338973 36
7 Станислав Мереминский 78.0392357550248 1584
8 Михаил Левандовский 77.8808484571701 1457
9 Сергей Волков 76.37698749483361 77
10 Дмитрий Кудинов 74.93726126438064 45
11 Алексей Гилёв 73.6842029756759 4306
12 Ирина Прокофьева 73.49193046962107 1065
13 Сусанна Бровер 73.41477174144568 600
14 Игорь Мокин 73.26092195385631 1176
15 Сергей Спешков 72.24009383626424 3737
16 Александр Коробейников 71.55318786156187 1401
17 Илья Новиков 70.81862176661939 1589
18 Светлана Бомешко 69.90417582059166 405
19 Михаил Царёв 69.82162136296171 501
20 Мария Кленницкая 69.69697170743376 1172
21 Александр Либер 69.29318695610917 3751
22 Михаил Матвеев 69.0682776845521 1025
23 Александр Марков 68.72240005128162 2903
24 Сергей Николенко 68.62123866240458 2039
25 Михаил Н

###### Вариант 2 тоже помог и топов стало еще больше в рейтинге

In [21]:
top_comps = get_top_comps(model.comp_probs, 20, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 92.87706194148954
2 Синхрон высшей лиги Москвы 75.54441535429417
3 Первенство правого полушария 67.8951670232854
4 Воображаемый музей 67.7316249266879
5 Записки охотника 60.11667672484285
6 Ускользающая сова 59.055236920491346
7 Знание – Сила VI 58.42746081506281
8 Зеркало мемориала памяти Михаила Басса 54.37212818503227
9 Чемпионат Минска. Лига А. Тур четвёртый 53.69211597499818
10 Чемпионат Мира. Этап 2 Группа С 53.131082505061016
11 Чемпионат России 50.79452129751421
12 Кубок городов 50.64117208681194
13 Чемпионат Мира. Этап 2. Группа В 50.31054943945977
14 Чемпионат Мира. Финал. Группа С 49.32284496366365
15 Тихий Донец: омут первый 49.002063659018724
16 Чемпионат Мира. Этап 3. Группа В 48.3846087218559
17 Чемпионат Мира. Этап 2. Группа А 47.92089333060703
18 Чемпионат Мира. Этап 3. Группа С 47.80652887667527
19 Львов зимой. Адвокат 47.584014425821515
20 Серия Premier. Седьмая печать 47.13917276034387
21 All Cats Are Beautiful 46.81082653395127
22 Антибинго 46.7064463

## 7

Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей
модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг
вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь
возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем
этого добиться:
2
- если хватит вычислительных ресурсов, сначала сделайте baseline совсем без
таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не
только на турнирах 2019 года; улучшилось ли качество предсказаний на 2020?
- одну схему со временем мы уже использовали: брали для обучения только
последний год турниров; примерно так делают, например, в теннисной
чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки
(например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
- предложите варианты базовой модели или алгоритма её обучения, которые
могли бы реализовать изменения рейтинга со временем; если получится,
реализуйте их на практике, проверьте, улучшатся ли предсказания на 2020.

Обучим модель с 2010 года. Модель использовал с тем же параметром q, но без домножения на количество сыгранных вопросов, поэтому качество будем сверять с качеством предпоследней модели

In [29]:
train_ids = get_ids_by_date(list(range(2010, 2020)), tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [30]:
model = EMModel(train_ids, results, q=1.5, force=False)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force-from2010")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(10):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 3328/3328 [00:39<00:00, 84.61it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.713384471362681
Kendall 0.5387065155768287




100%|██████████| 3328/3328 [00:39<00:00, 84.15it/s] 


Iteration 2
Spearman 0.7882458511158399
Kendall 0.6073122546677465




100%|██████████| 3328/3328 [00:40<00:00, 83.08it/s] 


...

Iteration 8
Spearman 0.8108786957236008
Kendall 0.6364129728774449




100%|██████████| 3328/3328 [00:39<00:00, 83.46it/s] 


Iteration 9
Spearman 0.8111544348894864
Kendall 0.6374032735137523




100%|██████████| 3328/3328 [00:39<00:00, 84.21it/s] 


Iteration 10
Spearman 0.8118073164392172
Kendall 0.6383330896894246




Аналогичная модель, обученная только на 2019 показала результат по Спирмену 0.821, по Кендаллу 0.645, что лучше чем результат модели, обученной на данных с 2010 года.

Я могу предложить брать последние N (наример 100) турниров за последние 2 или 3 года.

In [33]:
train_ids = get_ids_by_date(list(range(2015, 2020)), tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [34]:
model = EMModel(train_ids, results, q=1.5, force=False, cut_num=100)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force-and_cut_from2010")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 2575/2575 [00:31<00:00, 80.82it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7481907661453886
Kendall 0.5703794399661397




100%|██████████| 2575/2575 [00:32<00:00, 79.31it/s] 


Iteration 2
Spearman 0.8035594294894245
Kendall 0.623334225772593




100%|██████████| 2575/2575 [00:31<00:00, 82.63it/s] 


...

Iteration 13
Spearman 0.8213068007879504
Kendall 0.6482510081214571




100%|██████████| 2575/2575 [00:31<00:00, 82.22it/s] 


Iteration 14
Spearman 0.8216899840494024
Kendall 0.6485295711142993




100%|██████████| 2575/2575 [00:31<00:00, 82.91it/s] 


Iteration 15
Spearman 0.8222108221805607
Kendall 0.6490202478117657




Текущая версия показывает метрики Спирман-0.8210, Кендалл-0.648 на 10-итерации, что чуть лучше модели обученной на данных 2019 года

In [35]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Евгений Поникаров 8.90156348191518 36
2 Евгений Аренгауз 7.865667292235821 45
3 Максим Руссо 7.173189048341509 12816
4 Илья Леошкевич 6.921314320175525 30
5 Ирина Прокофьева 6.0854712966896605 5216
6 Иван Семушин 5.60938394177824 18139
7 Евгений Васильев 5.570340307264813 36
8 Артём Богданович 5.393871346397975 36
9 Александра Брутер 5.353421777750096 16289
10 Валентина Подюкова 5.316847417823507 36
11 Асель Куспанова 5.252031061294424 70
12 Елена Чигодайкина 5.250310389220463 73
13 Михаил Левандовский 5.165867595822574 7646
14 Илья Новиков 5.164581267989783 9922
15 Артём Сорожкин 5.12754213130841 19156
16 Станислав Мереминский 5.121706446048719 12420
17 Лада Латышева 5.034077898412492 36
18 Михаил Савченков 5.004114636309374 16903
19 Владимир Брайман 5.003611414728699 307
20 Антон Губанов 4.901856169157095 334
21 Александр Аникин 4.866395077046838 42
22 Александр Коробейников 4.788297527002478 9201
23 Владимир Тегин 4.727049589797926 28
24 Евгения Шатохина 4.713116462611123 36
25 Се

###### Скорее всего при расчете формул, либо их реализации, у меня могут быть ошибки. Думаю сделал все что мог на данный момент. Возможно стоило делать модели на основе линейной или логистической регрессии, но я поначалу не сообразил как, потом было поздно)